# Interfaces con otros lenguajes: C

Existen varias formas de utilizar bibliotecas o códigos hechos en C desde Python. Nosotros veremos el uso de `Ctypes`, sin embargo existen otras alternativas como  [Cython](https://cython.org/), [CFFI](https://cffi.readthedocs.io/en/latest/), [pybind11](https://pybind11.readthedocs.io/en/stable/) y [Boost.Python](https://www.boost.org/doc/libs/1_70_0/libs/python/doc/html/index.html). 


## Ejemplo 1: Problema a resolver

Supongamos que queremos resolver el problema de la rotación de vectores en el espacio usando los tres ángulos de Euler.



In [1]:
import numpy as np

In [2]:
pwd

'/Users/flavioc/Library/Mobile Documents/com~apple~CloudDocs/Documents/cursos/curso-python'

Si ya tenemos un módulo donde están programadas las funciones necesarias 

In [1]:
# %load rotacion_p.py
#! /usr/bin/ipython3
import numpy as np


def matrix_rotation(angles):
  cx, cy, cz = np.cos(angles)
  sx, sy, sz = np.sin(angles)
  R = np.zeros((3, 3))
  R[0, 0] = cx * cz - sx * cy * sz
  R[0, 1] = cx * sz + sx * cy * cz
  R[0, 2] = sx * sy

  R[1, 0] = -sx * cz - cx * cy * sz
  R[1, 1] = -sx * sz + cx * cy * cz
  R[1, 2] = cx * sy

  R[2, 0] = sy * sz
  R[2, 1] = -sy * cz
  R[2, 2] = cy
  return R


def rotate(angles, v):
  return np.dot(matrix_rotation(angles), v)


es fácil utilizarlas. Las importamos y utilizamos

In [2]:
# import rotacion_p as rotp
N = 100
# Ángulos de Euler
angle = np.random.random(3)
# Definimos N vectores tridimensionales
v = np.random.random((3, N))

In [3]:
# y= rotp.rotate(angle, v)
y = rotate(angle,v)

In [4]:
print(angle)
print(y[:,0:5].T)

[0.73462339 0.82242416 0.06741216]
[[ 0.59011544  0.61175303  0.54300288]
 [ 0.48735192  0.31644157 -0.54454539]
 [ 0.67186984  0.59029379  0.37867639]
 [ 0.55542883 -0.27666135 -0.16133539]
 [ 0.94544654 -0.12498498 -0.09458971]]


## Interfaces con C

Veamos cómo trabajar si tenemos el código para realizar las rotaciones en C.

### Primer ejemplo: Nuestro código

El código en C que tenemos es:

```c
 typedef struct {
      float m[3][3];
    } m3x3;

    typedef struct {
      float a[3];
    } v3;
    
...

float * rotate(float angles[3], float *v, int N){

        m3x3 R = matrix_rotation(angles);
        
        float* y = (float*)malloc(3*N*sizeof(float));
        v3 p;

        printf("%p\n",y);
        for(int i=0; i<N; i++){
            // p = &y[i*3];
            p = matmul3(R,&v[i*3]);
            y[i*3+0] = p.a[0];
            y[i*3+1] = p.a[1];
            y[i*3+2] = p.a[2];
            // printf("%6.3f %6.3f %6.3f \n",y[i*3+0],y[i*3+1],y[i*3+2]);
        }
        return y;


  }

```

In [5]:
cd interfacing_C

/Users/flavioc/Library/Mobile Documents/com~apple~CloudDocs/Documents/cursos/curso-python/interfacing_C


In [8]:
!cat rotacion.c

#include<stdio.h>
#include<stdlib.h>
#include<math.h> 


    typedef struct {
      float m[3][3];
    } m3x3;

    typedef struct {
      float a[3];
    } v3;

  // !> matrix_rotation
  // !! Crea la matriz de rotación correspondiente a los tres ángulos de Euler
  // !! 
  // !! @param angles 
  // !! @return R

    m3x3 matrix_rotation(float angles[3]){

        m3x3 R;

        float cx = cos(angles[0]); 
        float cy = cos(angles[1]); 
        float cz = cos(angles[2]);
        float sx = sin(angles[0]); 
        float sy = sin(angles[1]); 
        float sz = sin(angles[2]);


        R.m[0][0] = cx*cz - sx*cy*sz;
        R.m[0][1] = cx*sz + sx*cy*cz;
        R.m[0][2] = sx*sy;

        R.m[1][0] = -sx*cz - cx*cy*sz;
        R.m[1][1] = -sx*sz + cx*cy*cz;
        R.m[1][2] = cx*sy;

        R.m[2][0] = sy*sz;
        R.m[2][1] = -sy*cz;
        R.m[2][2] = cy;

        return R;
  }

  v3 matmul3(m3x3 A, float b[3]){

      static v3 a;

      for(int i=0; i < 3; i++){
       

### CTypes

No vamos a usar directamente `Ctypes`, sino a través de `NumPy`, que provee algunas funciones convenientes para acceder al código C.

El primer paso es compilar nuestro código y generar una biblioteca:
```bash
$ gcc -fpic -Wall -shared rotacion.c -o librotacion.so
```

Si uno trabaja en Windows, generará una dll

```cmd
cl.exe -c rotacion.c
link.exe /DLL /OUT:rotacion.dll
```


In [6]:
!gcc -fpic -Wall -shared rotacion.c -o librotacion.so

In [7]:
!ls

librotacion.so rotacion.c


En segundo lugar, importamos el módulo `ctypeslib`

In [8]:
import numpy.ctypeslib as ctl

Este módulo nos provee de la función `load_library` para importar la biblioteca

In [9]:
help(ctl.load_library)

Help on function load_library in module numpy.ctypeslib:

load_library(libname, loader_path)
    It is possible to load a library using
    >>> lib = ctypes.cdll[<full_path_name>] # doctest: +SKIP
    
    But there are cross-platform considerations, such as library file extensions,
    plus the fact Windows will just load the first library it finds with that name.
    NumPy supplies the load_library function as a convenience.
    
    Parameters
    ----------
    libname : str
        Name of the library, which can have 'lib' as a prefix,
        but without an extension.
    loader_path : str
        Where the library can be found.
    
    Returns
    -------
    ctypes.cdll[libpath] : library object
       A ctypes library object
    
    Raises
    ------
    OSError
        If there is no library with the expected extension, or the
        library is defective and cannot be loaded.



In [11]:
rotc = ctl.load_library('librotacion.so','.')

Una vez cargada la biblioteca, tenemos que definir adecuadamente cómo pasar los argumentos a la función `rotate` de C:

```C
    float * rotate(float angles[3], float *v, int N)
```

Para eso se utiliza la función `argtypes` que recibe una lista de tipos. Notemos que los dos primeros argumentos son arreglos de C (o sea, punteros), mientras que el último es un entero.

In [12]:
npflags = ['C_CONTIGUOUS']   # Require a C contiguous array in memory

float_1d_type = ctl.ndpointer(dtype=np.float32, ndim=1, flags=npflags) # Puntero a float, 1D
float_2d_type = ctl.ndpointer(dtype=np.float32, ndim=2, flags=npflags) # Puntero a float, 2D

In [13]:
type(float_1d_type)

_ctypes.PyCSimpleType

Con estos tipos de datos, defino los tipos de argumentos, que son tres en total. El último es un dato de tipo entero, para lo cual se usa directamente `c_intp`.

In [29]:
rotc.rotate.argtypes =  [float_1d_type, float_2d_type, ctl.c_intp]

Hagamos un ejemplo sencillo con N=2

In [30]:
# import rotacion_p as rotp
N = 2
# Ángulos de Euler
angle = np.random.random(3).astype(np.float32)
# Definimos N vectores tridimensionales
v = np.random.random((3, N)).astype(np.float32)

Las funciones que dispongo en C reciben tipos `float`, es decir que me tengo que asegurar esto a través del método `astype`.

Ahora tenemos que definir el tipo de dato de salida, que retorna C a través de un puntero a float, `float*`. Para esto usamos el método `restype`. Como a priori no sé qué tipo de rango tiene mi arreglo de salida, tengo que definirlo explícitamente. 

In [31]:
rotc.rotate.restype = ctl.ndpointer(dtype=np.float32, shape=(N,3)) 

Hay que tener precaución con el manejo de arreglos, que es muy distinto en C y Python. En Python son objetos, de los cuales yo puedo tener distintas vistas, slices, etc. Hay que recordar que en principio estas son formas de acceder al mismo objeto, pero no se pueden traducir directamente a C, que necesita un arreglo contiguo de datos.

In [32]:
v = np.array([[1,0], [0,1], [0,0]]).astype(np.float32) 
vt = v.T.copy()

print(np.shape(v))
print(np.shape(v.T))

(3, 2)
(2, 3)


Veamos, v es un arreglo de 3 filas y 2 columnas, que contiene *dos* vectores de tres dimensiones que se desean rotar, organizados como columnas. Esto *no* es lo que necesita mi arreglo en C, que es tiene los vectores organizados contiguamente en un solo arreglo unidimensional. Entonces, tengo que transformarlo. Para eso usamos el `.T`. Ojo que además, hay que crear un objeto nuevo con `copy()`, sino es una vista del mismo objeto `v`.

In [33]:
angle90 = np.array([0,0,np.pi/2],dtype = np.float32)
print(angle90)

[0.        0.        1.5707964]


In [34]:
yf = rotc.rotate(angle90,
                      vt,
                      N) 
y = rotate(angle90,v)

In [36]:
np.set_printoptions(suppress=True)

print(y)
print(yf.T)
np.allclose(y,yf.T)

[[-0.00000004  1.        ]
 [-1.         -0.00000004]
 [ 0.          0.        ]]
[[-0.00000004  1.        ]
 [-1.         -0.00000004]
 [ 0.          0.        ]]


True